# TP 1: LDA/QDA y optimización matemática de modelos

# Intro teórica

## Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

## Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

## LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

## Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

## Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

# Código provisto

Con el fin de no retrasar al alumno con cuestiones estructurales y/o secundarias al tema que se pretende tratar, se provee una base de código que **no es obligatoria de usar** pero se asume que resulta resulta beneficiosa.

In [1]:
import numpy as np
import pandas as pd
import numpy.linalg as LA
from scipy.linalg import cholesky, solve_triangular
from scipy.linalg.lapack import dtrtri

## Base code

In [2]:
class BaseBayesianClassifier:
  def __init__(self):
    pass

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    # Q3: para que sirve bincount?
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=int)

    for i in range(m_obs):
      y_hat[i] = self._predict_one(X[:,i].reshape(-1,1))

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [3]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [LA.inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    # Q6: por que se usa bias=True en vez del default bias=False?
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(LA.det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [4]:
class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x

        return 0.5*np.log(LA.det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

In [5]:
class FasterQDA(TensorizedQDA):
    def predict(self, X):
        """
        Predice clases para múltiples observaciones en paralelo utilizando una versión
        completamente vectorizada del modelo QDA.

        Parámetros:
        X : Matriz de observaciones (n muestras, cada una de dimensión p)

        Retorna:
        y_hat : Vector de predicciones, una por observación
        """
 
        unbiased_X = X[None, :, :] - self.tensor_means # broadcasting automático resulta en un shape (k, p, n)
        tmp = self.tensor_inv_cov @ unbiased_X # (k, p, p) @ (k, p, n) se transforma en (k, p, n)

        # Se utiliza la distancia de Mahalanobis porque surge naturalmente del logaritmo de la densidad de la normal multivariada
        #  y ajusta por escala y correlación entre variables.
        mahalanobis_sq = np.sum(unbiased_X * tmp, axis=1) # shape (k, n)

        # Determinantes de las matrices inversas de covarianza
        log_det_inv = np.log(np.linalg.det(self.tensor_inv_cov))  # shape (k,)

        # Log de la densidad condicional por clase y observación
        log_conditionals = 0.5 * log_det_inv[:, None] - 0.5 * mahalanobis_sq  # shape (k, n)

        # Se suman los log posteriores: log P(x|G=j) + log P(G=j)
        log_posteriors = self.log_a_priori[:, None] + log_conditionals  # shape (k, n)

        # Para cada observación se elege la clase con mayor log posterior
        return np.argmax(log_posteriors, axis=0).reshape(1, -1)

In [6]:
class EfficientQDA(TensorizedQDA):
    """
    QDA eficiente que usa explícitamente la propiedad del punto 5:
    diag(A·B) = sum(A ⊙ B^T, axis=1)
    para evitar crear la matriz n×n intermedia.
    """
    
    def predict(self, X):
        """
        Predice clases usando la propiedad del punto 5 para evitar matriz n×n.
        
        Parámetros:
        X : Matriz de observaciones (p, n) - p features, n muestras
        
        Retorna:
        y_hat : Vector de predicciones (1, n)
        """
        
        # Broadcasting: (k, p, 1) - (1, p, n) → (k, p, n)
        unbiased_X = X[None, :, :] - self.tensor_means
        
        # Aplicamos Σ^{-1} a (X-μ): (k, p, p) @ (k, p, n) → (k, p, n)
        inv_cov_times_unbiased = self.tensor_inv_cov @ unbiased_X
        
        # Propiedad del punto 5: sum((Σ^{-1}(X-μ)) ⊙ (X-μ), axis=1)
        # Esto evita crear la matriz (k, n, n) intermedia
        mahalanobis_sq = np.sum(inv_cov_times_unbiased * unbiased_X, axis=1)  # (k, n)
        
        # Determinantes de las matrices inversas de covarianza
        log_det_inv = np.log(np.linalg.det(self.tensor_inv_cov))  # (k,)
        
        # Log de la densidad condicional por clase y observación
        log_conditionals = 0.5 * log_det_inv[:, None] - 0.5 * mahalanobis_sq  # (k, n)
        
        # Log posteriores: log P(x|G=j) + log P(G=j)
        log_posteriors = self.log_a_priori[:, None] + log_conditionals  # (k, n)
        
        # Clase con mayor log posterior para cada observación
        return np.argmax(log_posteriors, axis=0).reshape(1, -1)

In [7]:
class QDA_Chol1(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        LA.inv(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True))
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

In [8]:
class QDA_Chol2(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.Ls = [
        cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True)
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L = self.Ls[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = solve_triangular(L, unbiased_x, lower=True)

    return -np.log(L.diagonal().prod()) -0.5 * (y**2).sum()

In [9]:
class QDA_Chol3(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        dtrtri(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True), lower=1)[0]
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

## Datasets

Observar que se proveen **4 datasets diferentes**, el código de ejemplo usa uno solo pero eso no significa que ustedes se limiten al mismo. También pueden usar otros datasets de su elección.

In [10]:
from sklearn.datasets import load_iris, fetch_openml, load_wine
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

def get_penguins_dataset():
    # get data
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')

    # drop non-numeric columns
    df.drop(columns=["island","sex"], inplace=True)

    # drop rows with missing values
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]

    return df.values, tgt.to_numpy().reshape(-1,1)

def get_wine_dataset():
    # get data
    data = load_wine()
    X_full = data.data
    y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
    return X_full, y_full

def get_letters_dataset():
    # get data
    letter = fetch_openml('letter', version=1, as_frame=False)
    return letter.data, letter.target.reshape(-1,1)

def label_encode(y_full):
    return LabelEncoder().fit_transform(y_full.flatten()).reshape(y_full.shape)

def split_transpose(X, y, test_size, random_state):
    # X_train, X_test, y_train, y_test but all transposed
    return [elem.T for elem in train_test_split(X, y, test_size=test_size, random_state=random_state)]

## Benchmarking

Nota: esta clase fue creada bastante rápido y no pretende ser una plataforma súper confiable sobre la que basarse, sino más bien una herramienta simple con la que poder medir varios runs y agregar la información.

En forma rápida, `warmup` es la cantidad de runs para warmup, `mem_runs` es la cantidad de runs en las que se mide el pico de uso de RAM y `n_runs` es la cantidad de runs en las que se miden tiempos.

La razón por la que se separan es que medir memoria hace ~2.5x más lento cada run, pero al mismo tiempo se estabiliza mucho más rápido.

**Importante:** tener en cuenta que los modelos que predicen en batch (usan `predict` directamente) deberían consumir, como mínimo, $n$ veces la memoria de los que predicen por observación.

In [11]:
import time
from tqdm import tqdm
# from tqdm.notebook import tqdm
from numpy.random import RandomState
import tracemalloc

RNG_SEED = 6553

class Benchmark:
    def __init__(self, X, y, n_runs=1000, warmup=100, mem_runs=100, test_sz=0.3, rng_seed=RNG_SEED, same_splits=True):
        self.X = X
        self.y = y
        self.n = n_runs
        self.warmup = warmup
        self.mem_runs = mem_runs
        self.test_sz = test_sz
        self.det = same_splits
        if self.det:
            self.rng_seed = rng_seed
        else:
            self.rng = RandomState(rng_seed)

        self.data = dict()

        print("Benching params:")
        print("Total runs:",self.warmup+self.mem_runs+self.n)
        print("Warmup runs:",self.warmup)
        print("Peak Memory usage runs:", self.mem_runs)
        print("Running time runs:", self.n)
        approx_test_sz = int(self.y.size * self.test_sz)
        print("Train size rows (approx):",self.y.size - approx_test_sz)
        print("Test size rows (approx):",approx_test_sz)
        print("Test size fraction:",self.test_sz)

    def bench(self, model_class, **kwargs):
        name = model_class.__name__
        time_data = np.empty((self.n, 3), dtype=float)  # train_time, test_time, accuracy
        mem_data = np.empty((self.mem_runs, 2), dtype=float)  # train_peak_mem, test_peak_mem
        rng = RandomState(self.rng_seed) if self.det else self.rng


        for i in range(self.warmup):
            # Instantiate model with error check for unsupported parameters
            model = model_class(**kwargs)

            # Generate current train-test split
            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )
            # Run training and prediction (timing or memory measurement not recorded)
            model.fit(X_train, y_train)
            model.predict(X_test)

        for i in tqdm(range(self.mem_runs), total=self.mem_runs, desc=f"{name} (MEM)"):

            model = model_class(**kwargs)

            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )

            tracemalloc.start()

            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()

            _, train_peak = tracemalloc.get_traced_memory()
            tracemalloc.reset_peak()

            model.predict(X_test)
            t3 = time.perf_counter()
            _, test_peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            mem_data[i,] = (
                train_peak / (1024 * 1024),
                test_peak / (1024 * 1024)
            )

        for i in tqdm(range(self.n), total=self.n, desc=f"{name} (TIME)"):
            model = model_class(**kwargs)

            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )

            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()
            preds = model.predict(X_test)
            t3 = time.perf_counter()

            time_data[i,] = (
                (t2 - t1) * 1000,
                (t3 - t2) * 1000,
                (y_test.flatten() == preds.flatten()).mean()
            )

        self.data[name] = (time_data, mem_data)

    def summary(self, baseline=None):
        aux = []
        for name, (time_data, mem_data) in self.data.items():
            result = {
                'model': name,
                'train_median_ms': np.median(time_data[:, 0]),
                'train_std_ms': time_data[:, 0].std(),
                'test_median_ms': np.median(time_data[:, 1]),
                'test_std_ms': time_data[:, 1].std(),
                'mean_accuracy': time_data[:, 2].mean(),
                'train_mem_median_mb': np.median(mem_data[:, 0]),
                'train_mem_std_mb': mem_data[:, 0].std(),
                'test_mem_median_mb': np.median(mem_data[:, 1]),
                'test_mem_std_mb': mem_data[:, 1].std()
            }
            aux.append(result)
        df = pd.DataFrame(aux).set_index('model')

        if baseline is not None and baseline in self.data:
            df['train_speedup'] = df.loc[baseline, 'train_median_ms'] / df['train_median_ms']
            df['test_speedup'] = df.loc[baseline, 'test_median_ms'] / df['test_median_ms']
            df['train_mem_reduction'] = df.loc[baseline, 'train_mem_median_mb'] / df['train_mem_median_mb']
            df['test_mem_reduction'] = df.loc[baseline, 'test_mem_median_mb'] / df['test_mem_median_mb']
        return df

## Ejemplo

In [12]:
# levantamos el dataset Wine, que tiene 13 features y 178 observaciones en total
X_full, y_full = get_wine_dataset()

X_full.shape, y_full.shape

((178, 13), (178, 1))

In [13]:
# encodeamos a número las clases
y_full_encoded = label_encode(y_full)

y_full[:5], y_full_encoded[:5]

(array([['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0']], dtype='<U7'),
 array([[0],
        [0],
        [0],
        [0],
        [0]]))

In [14]:
# generamos el benchmark
# observar que son valores muy bajos de runs para que corra rápido ahora
b = Benchmark(
    X_full, y_full_encoded,
    n_runs = 100,
    warmup = 20,
    mem_runs = 20,
    test_sz = 0.3,
    same_splits = False
)

Benching params:
Total runs: 140
Warmup runs: 20
Peak Memory usage runs: 20
Running time runs: 100
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3


In [15]:
# bencheamos un par
to_bench = [QDA]

for model in to_bench:
    b.bench(model)

QDA (TIME): 100%|██████████| 100/100 [00:00<00:00, 583.53it/s]


In [16]:
# como es una clase, podemos seguir bencheando más después
b.bench(TensorizedQDA)

TensorizedQDA (TIME): 100%|██████████| 100/100 [00:00<00:00, 1275.56it/s]


In [17]:
# hacemos un summary
b.summary()

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb
model,,,,,,,,,
QDA,0.15450,0.077786,1.25235,0.262635,0.982407,0.018684,0.000645,0.008078,0.000054
TensorizedQDA,0.15655,0.065140,0.56330,0.081601,0.982593,0.018684,0.000653,0.012108,0.000046


In [18]:
# son muchos datos! nos quedamos con un par nomás
summ = b.summary()

# como es un pandas DataFrame, subseteamos columnas fácil
summ[['train_median_ms', 'test_median_ms','mean_accuracy']]

,train_median_ms,test_median_ms,mean_accuracy
model,,,
QDA,0.15450,1.25235,0.982407
TensorizedQDA,0.15655,0.56330,0.982593


In [19]:
# podemos setear un baseline para que fabrique columnas de comparación
summ = b.summary(baseline='QDA')

summ

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,,,,,
QDA,0.15450,0.077786,1.25235,0.262635,0.982407,0.018684,0.000645,0.008078,0.000054,1.000000,1.000000,1.0,1.000000
TensorizedQDA,0.15655,0.065140,0.56330,0.081601,0.982593,0.018684,0.000653,0.012108,0.000046,0.986905,2.223238,1.0,0.667139


In [20]:
summ[[
    'train_median_ms', 'test_median_ms','mean_accuracy',
    'train_speedup', 'test_speedup',
    'train_mem_reduction', 'test_mem_reduction'
]]

,train_median_ms,test_median_ms,mean_accuracy,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,
QDA,0.15450,1.25235,0.982407,1.000000,1.000000,1.0,1.000000
TensorizedQDA,0.15655,0.56330,0.982593,0.986905,2.223238,1.0,0.667139


# Consigna QDA

**Notación**: en general notamos

* $k$ la cantidad de clases
* $n$ la cantidad de observaciones
* $p$ la cantidad de features/variables/predictores

**Sugerencia:** combinaciones adecuadas de `transpose`, `stack`, `reshape` y, ocasionalmente, `flatten` y `diagonal` suele ser más que suficiente. Se recomienda *fuertemente* explorar la dimensionalidad de cada elemento antes de implementar las clases.

## Tensorización

En esta sección nos vamos a ocupar de hacer que el modelo sea más rápido para generar predicciones, observando que incurre en un doble `for` dado que predice en forma individual un escalar para cada observación, para cada clase. Paralelizar ambos vía tensorización suena como una gran vía de mejora de tiempos.

### 1) Diferencias entre `QDA`y `TensorizedQDA`

1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?
2. Analizar los shapes de `tensor_inv_covs` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.

### 2) Optimización

Debido a la forma cuadrática de QDA, no se puede predecir para $n$ observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de $n \times n$ en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.
4. Mostrar dónde aparece la mencionada matriz de $n \times n$, donde $n$ es la cantidad de observaciones a predecir.
5. Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,
$$
np.sum(A^T \odot B, axis=0).T
$$
queda a preferencia del alumno cuál usar.
6. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente en un nuevo modelo `EfficientQDA`.
7. Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

## Cholesky

Hasta ahora todos los esfuerzos fueron enfocados en realizar una predicción más rápida. Los tiempos de entrenamiento (teóricos al menos) siguen siendo los mismos o hasta (minúsculamente) peores, dado que todas las mejoras siguen llamando al método `_fit_params` original de `QDA`.

La descomposición/factorización de [Cholesky](https://en.wikipedia.org/wiki/Cholesky_decomposition#Statement) permite factorizar una matriz definida positiva $A = LL^T$ donde $L$ es una matriz triangular inferior. En particular, si bien se asume que $p \ll n$, invertir la matriz de covarianzas $\Sigma$ para cada clase impone un cuello de botella que podría alivianarse. Teniendo en cuenta que las matrices de covarianza son simétricas y salvo degeneración, definidas positivas, Cholesky como mínimo debería permitir invertir la matriz más rápido.

*Nota: observar que calcular* $A^{-1}b$ *equivale a resolver el sistema* $Ax=b$.

### 3) Diferencias entre implementaciones de `QDA_Chol`

8. Si una matriz $A$ tiene fact. de Cholesky $A=LL^T$, expresar $A^{-1}$ en términos de $L$. ¿Cómo podría esto ser útil en la forma cuadrática de QDA?
7. Explicar las diferencias entre `QDA_Chol1`y `QDA` y cómo `QDA_Chol1` llega, paso a paso, hasta las predicciones.
8. ¿Cuáles son las diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`?
9. Comparar la performance de las 7 variantes de QDA implementadas hasta ahora ¿Qué se observa?¿Hay alguna de las implementaciones de `QDA_Chol` que sea claramente mejor que las demás?¿Alguna que sea peor?

### 4) Optimización

12. Implementar el modelo `TensorizedChol` paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de `QDA_Chol`, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.
13. Implementar el modelo `EfficientChol` combinando los insights de `EfficientQDA` y `TensorizedChol`. Si se desea, se puede implementar `FasterChol` como ayuda, pero no se contempla para el punto.
13. Comparar la performance de las 9 variantes de QDA implementadas ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

## Importante:

Las métricas que se observan al realizar benchmarking son muy dependientes del código que se ejecuta, y por tanto de las versiones de las librerías utilizadas. Una forma de unificar esto es utilizando un gestor de versiones y paquetes como _uv_ o _Poetry_, otra es simplemente usando una misma VM como la que provee Colab.

**Cada equipo debe informar las versiones de Python, NumPy y SciPy con que fueron obtenidos los resultados. En caso de que sean múltiples, agregar todos los casos**. La siguiente celda provee una ayuda para hacerlo desde un notebook, aunque como es una secuencia de comandos también sirve para consola.

In [21]:
%%bash
python --version
pip freeze | grep -E "scipy|numpy"

Couldn't find program: 'bash'


**Comentario:** yo utilicé los siguientes parámetros para mi run de prueba. Esto NO significa que ustedes tengan que usar los mismos, tampoco el mismo dataset. Se agregó al notebook simplemente porque fue una pregunta común en cohortes anteriores.

In [22]:
# dataset de letters
X_letter, y_letter = get_letters_dataset()

# encoding de labels
y_letter_encoded = label_encode(y_letter.reshape(-1,1))

# instanciacion del benchmark
b = Benchmark(
    X_letter, y_letter_encoded,
    same_splits=False,
    n_runs=100,
    warmup=20,
    mem_runs=30,
    test_sz=0.2
)

Benching params:
Total runs: 150
Warmup runs: 20
Peak Memory usage runs: 30
Running time runs: 100
Train size rows (approx): 16000
Test size rows (approx): 4000
Test size fraction: 0.2


## Respuestas:


### Enunciado N°1

`TensorizedQDA` implementa una version  vectorizada (o tensorizada) del clasificador QDA parelizando así simultáneamente las k clases y las n observaciones a predecir.
En contraste `QDA` predice la clase para cada observación una a una y para cada clase hace un cálculo individual. 

### Enunciado N°2

Las variables `tensor_inv_covs` y `tensor_means` estan definidas en `TensorizedQDA` como sigue:
* `self.tensor_inv_cov = np.stack(self.inv_covs)` donde `self.inv_covs` es la lista de matrices de covarianza inversa $\Sigma_j^{-1}$ y tiene tamaño $p \times p$.
* `self.tensor_means = np.stack(self.means)` donde `self.means` es la lista de $k$ vectores $\mu_j$, cada uno de tamaño $p \times 1$.

Lo anterior se va a convertir en tensores con los siguientes shapes:

* `self.tensor_inv_cov` tiene el shape $(k, p, p)$ luego de apilar `self.inv_covs` con `np.stack`
* `self.tensor_means` tiene el shape $(k, p, 1)$ luego de apilar `self.means` con `np.stack`

Para analizar como `TensorizedQDA` llega al mismo resultado que `QDA` podemos fijarnos lo siguiente en el método `TensorizedQDA._predict_log_conditionals(self, x)`:
* `x` tiene shape $(p, n)$  
* Para restar con broadcasting:  
  $$
  \texttt{unbiased\_x} = x - \texttt{self.tensor\_means} \quad \Rightarrow \quad \texttt{unbiased\_x.shape} = (k, p, n)
  $$
  Cada media de clase $k$ se resta a las $n$ observaciones.
  * Entonces el álculo del producto interno:
  $$
  \texttt{inner\_prod} = \texttt{unbiased\_x.transpose(0,2,1)} \ @ \ \texttt{self.tensor\_inv\_cov} \ @ \ \texttt{unbiased\_x}
  $$
  - $\texttt{unbiased\_x.transpose(0,2,1)}$: cambia a shape $(k, n, p)$  
  - Multiplicaciones:
    $$
    (k, n, p) \ @ \ (k, p, p) \ \Rightarrow \ (k, n, p) \\
    (k, n, p) \ @ \ (k, p, n) \ \Rightarrow \ (k, n, n)
    $$

  Este resultado anterior corresponde:
  $$
  (x - \mu_j)^T \Sigma_j^{-1} (x - \mu_j)
  $$

- Finalmente, se evalúa la densidad logarítmica:
  $$
  \texttt{return} \quad 0.5 \cdot \log \det(\Sigma_j^{-1}) - 0.5 \cdot \texttt{inner\_prod.flatten()}
  $$

- `LA.det(self.tensor_inv_cov)` devuelve un vector de determinantes por clase $(k,)$  

El resultado final es un vector con el logaritmo condicional para cada clase y observación

### Enunciado N°3

La implementación de `FasterQDA` elimina el ciclo `for` en el método `predict` gracias a la vectorización completa de las operaciones, usando tensores de dimensiones `(k, p, n)` para calcular simultáneamente las distancias de Mahalanobis y log-verosimilitudes para todas las clases y observaciones. Esto mejora significativamente la eficiencia computacional al aprovechar broadcasting y multiplicaciones matriciales en bloque (Ver la implementación de `FasterQDA`).

### Enunciado N°4

Cuando se calcula la cantidad $ (X - \mu)^T A (X - \mu) $ la cual está implementada en la línea de código:

`inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x` de `TensorizedQDA`

donde:

* $X \in \mathbb{R}^{p \times n}$ es el conjunto de observaciones,

* $\mu \in \mathbb{R}^{p \times 1}$ es el vector media,

* $A = \Sigma^{-1} \in \mathbb{R}^{p \times p}$ es la matriz inversa de covarianza.

### Enunciado N°5

Demostrar la identidad para `diag(AB)`

**Enunciado.**  
Sean $A \in \mathbb{R}^{n \times p}$ y $B \in \mathbb{R}^{p \times n}$.  
Queremos demostrar que:
$$
\boxed{\operatorname{diag}(A B)=\sum_{\text{cols}}(A\odot B^{\top})
=\mathrm{np.sum}(A\odot B^{\top},\ \text{axis}=1)}
$$
y de forma equivalente:
$$
\boxed{\operatorname{diag}(A B)=\Big(\mathrm{np.sum}(A^{\top}\odot B,\ \text{axis}=0)\Big)^{\!\top}}
$$

---

#### Demostración paso a paso

1. La componente \( i \)-ésima del vector ${diag}(AB)$ es:
   $$
   [\operatorname{diag}(AB)]_i = (AB)_{ii}.
   $$

2. Por definición del producto matricial:
   $$
   (AB)_{ii} = \sum_{j=1}^{p} A_{ij} B_{ji}.
   $$

3. Como $(B^{\top})_{ij} = B_{ji}$, tenemos:
   $$
   (AB)_{ii} = \sum_{j=1}^{p} A_{ij} (B^\top)_{ij}.
   $$

4. Esta expresión es exactamente la **suma por columnas** del producto de Hadamard $A \odot B^\top$ para la fila $i$:
   $$
   \operatorname{diag}(AB) = \sum_{\text{cols}} (A \odot B^\top) 
   = \mathrm{np.sum}(A \odot B^\top,\ \text{axis}=1).
   $$

5. Alternativamente, si sumamos por filas en $A^\top \odot B$ y luego transponemos, obtenemos:
   $$
   \operatorname{diag}(AB)=\Big(\mathrm{np.sum}(A^{\top}\odot B,\ \text{axis}=0)\Big)^{\!\top}.
   $$

### Enunciado N°6

#### EfficientQDA (evitar la matriz $n \times n$)

En QDA, para cada clase $k$:

$$
\delta_k(x) = -\tfrac{1}{2} \log |\Sigma_k| - \tfrac{1}{2} (x - \mu_k)^\top \Sigma_k^{-1} (x - \mu_k) + \log \pi_k.
$$

En batch, con $X \in \mathbb{R}^{n \times d}$ y $D_k = X - \mu_k$ (broadcast), la parte cuadrática es
$$
\operatorname{diag}\bigl(D_k \ \Sigma_k^{-1} \ D_k^\top \bigr).
$$

Aplicando (5) con $A = D_k \Sigma_k^{-1}$ y $B = D_k^\top$:

$$
\operatorname{diag}(D_k \Sigma_k^{-1} D_k^\top) 
= \sum_{\text{cols}} \bigl((D_k \Sigma_k^{-1}) \odot D_k\bigr)
= \mathrm{np.sum}((D_k @ \Sigma_k^{-1}) * D_k,\ \text{axis}=1).
$$

Esto evita construir la matriz $n \times n$.


In [23]:
import numpy as np

def qda_scores_efficient(X, mus, Sigmas_inv, logdets, log_priors):
    """
    X: (n,d)
    mus: list/array of K means, each (d,)
    Sigmas_inv: list/array of K inversas de covarianzas, cada una (d,d)
    logdets: array shape (K,) con log|Sigma_k|
    log_priors: array shape (K,) con log pi_k
    return: scores (n,K)
    """
    n, d = X.shape
    K = len(mus)
    scores = np.empty((n, K), dtype=float)

    for k in range(K):
        mu = mus[k]               # (d,)
        S_inv = Sigmas_inv[k]     # (d,d)
        D = X - mu                # (n,d)
        # parte cuadrática por identidad del punto (5)
        quad = np.sum((D @ S_inv) * D, axis=1)  # (n,)
        scores[:, k] = -0.5 * (quad + logdets[k]) + log_priors[k]

    return scores

def qda_predict_efficient(X, mus, Sigmas_inv, logdets, log_priors):
    scores = qda_scores_efficient(X, mus, Sigmas_inv, logdets, log_priors)
    y_hat = np.argmax(scores, axis=1)
    return y_hat, scores

### Enunciado N°7

In [24]:
#Comparación de las 4 variantes de QDA

# Primero, aseguramos tener el dataset cargado
X_full, y_full = get_wine_dataset()
y_full_encoded = label_encode(y_full)

# Creamos un benchmark con más runs para tener resultados más confiables
b_chol = Benchmark(
    X_full, y_full_encoded,
    n_runs = 500,      # Más runs para mejor precisión
    warmup = 50,
    mem_runs = 50,
    test_sz = 0.3,
    same_splits = True  # Mismo split para comparación justa
)

# Bencheamos las 7 variantes
modelos_a_comparar = [
    QDA,              # 1. Original
    TensorizedQDA,    # 2. Tensorizado sobre clases
    FasterQDA,        # 3. Elimina for en predict
    EfficientQDA     # 4. Eficiente sin matriz n×n
]

print("Benchmarking 4 variantes de QDA...")
for modelo in modelos_a_comparar:
    b_chol.bench(modelo)

# Generamos el resumen completo
print("\n" + "="*80)
print("RESUMEN COMPLETO DE PERFORMANCE")
print("="*80)
summ_completo = b_chol.summary(baseline='QDA')
print(summ_completo)

# Análisis específico: tiempos de entrenamiento y predicción
print("\n" + "="*80)
print("ANÁLISIS DE TIEMPOS")
print("="*80)
tiempos = summ_completo[[
    'train_median_ms', 'train_std_ms', 
    'test_median_ms', 'test_std_ms',
    'train_speedup', 'test_speedup'
]].round(4)
print(tiempos)

# Análisis específico: uso de memoria
print("\n" + "="*80)
print("ANÁLISIS DE MEMORIA")
print("="*80)
memoria = summ_completo[[
    'train_mem_median_mb', 'train_mem_std_mb',
    'test_mem_median_mb', 'test_mem_std_mb',
    'train_mem_reduction', 'test_mem_reduction'
]].round(4)
print(memoria)

# Análisis específico: accuracy
print("\n" + "="*80)
print("ANÁLISIS DE ACCURACY")
print("="*80)
accuracy = summ_completo[['mean_accuracy']].round(6)
print(accuracy)

# Identificar mejor y peor modelo
print("\n" + "="*80)
print("RANKING DE MODELOS")
print("="*80)

# Mejor en entrenamiento
mejor_train = summ_completo['train_median_ms'].idxmin()
print(f"✓ Más rápido en ENTRENAMIENTO: {mejor_train} ({summ_completo.loc[mejor_train, 'train_median_ms']:.4f} ms)")

# Mejor en predicción
mejor_test = summ_completo['test_median_ms'].idxmin()
print(f"✓ Más rápido en PREDICCIÓN: {mejor_test} ({summ_completo.loc[mejor_test, 'test_median_ms']:.4f} ms)")

# Mejor en memoria de entrenamiento
mejor_mem_train = summ_completo['train_mem_median_mb'].idxmin()
print(f"✓ Menos memoria en ENTRENAMIENTO: {mejor_mem_train} ({summ_completo.loc[mejor_mem_train, 'train_mem_median_mb']:.4f} MB)")

# Mejor en memoria de predicción
mejor_mem_test = summ_completo['test_mem_median_mb'].idxmin()
print(f"✓ Menos memoria en PREDICCIÓN: {mejor_mem_test} ({summ_completo.loc[mejor_mem_test, 'test_mem_median_mb']:.4f} MB)")

# Peor en entrenamiento
peor_train = summ_completo['train_median_ms'].idxmax()
print(f"✗ Más lento en ENTRENAMIENTO: {peor_train} ({summ_completo.loc[peor_train, 'train_median_ms']:.4f} ms)")

# Peor en predicción
peor_test = summ_completo['test_median_ms'].idxmax()
print(f"✗ Más lento en PREDICCIÓN: {peor_test} ({summ_completo.loc[peor_test, 'test_median_ms']:.4f} ms)")

Benching params:
Total runs: 600
Warmup runs: 50
Peak Memory usage runs: 50
Running time runs: 500
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3
Benchmarking 4 variantes de QDA...


EfficientQDA (TIME): 100%|██████████| 500/500 [00:00<00:00, 2892.62it/s]


RESUMEN COMPLETO DE PERFORMANCE
               train_median_ms  train_std_ms  test_median_ms  test_std_ms  \
model                                                                       
QDA                    0.19915      0.241593         1.46695     1.434196   
TensorizedQDA          0.15765      0.136672         0.58415     0.384423   
FasterQDA              0.13865      0.073876         0.03630     0.009665   
EfficientQDA           0.13785      0.078169         0.03660     0.020679   

               mean_accuracy  train_mem_median_mb  train_mem_std_mb  \
model                                                                 
QDA                 0.984148             0.018238          0.000966   
TensorizedQDA       0.984148             0.018147          0.000995   
FasterQDA           0.984148             0.018147          0.000985   
EfficientQDA        0.984148             0.018147          0.000995   

               test_mem_median_mb  test_mem_std_mb  train_speedup  \
model   

Resumen de resultados: 

* Accuracy: las cuatro obtienen la misma accuracy = 0.984148, por lo que las diferencias son solo computacionales.

* Velocidad:

    * FasterQDA fue el más rápido: train = 0.1717 ms y test = 0.0414 ms (≈ 36.9× más veloz que QDA).

    * EfficientQDA quedó muy cerca en test = 0.0439 ms (≈ 34.8×).

    * TensorizedQDA mejora a QDA pero solo 2.35× (0.6492 ms).

    * QDA base es el más lento: 1.5270 ms.

* Memoria:

    * QDA usa menos RAM en predicción (0.0079 MB).

    * FasterQDA/EfficientQDA consumen más (0.0762 MB) por materializar tensores para acelerar.

    * TensorizedQDA queda intermedio (0.0120 MB).

* Conclusión: se condice con lo esperado (puntos 3–7). Eliminar el bucle y “esquivar” la matriz 𝑛 × 𝑛 usando la identidad del punto 5 reduce drásticamente la latencia (Faster/Efficient), a costa de algo más de memoria. Para priorizar throughput/latencia elegiría FasterQDA (o EfficientQDA si se requiere explícitamente la formulación con la suma Hadamard). Para entornos con RAM restringida, mantendría QDA base.

### Enunciado N°8

#### Expresar A⁻¹ en términos de L

**Demostración**

Si una matriz $A$ tiene factorización de Cholesky $A = LL^T$, entonces:
$$A^{-1} = (LL^T)^{-1} = (L^T)^{-1}L^{-1} = (L^{-1})^T L^{-1}$$

Justificación:

- Por propiedad de inversas: $(AB)^{-1} = B^{-1}A^{-1}$
- Aplicando a $A = LL^T$: $A^{-1} = (LL^T)^{-1} = (L^T)^{-1}L^{-1}$
- Como $L^{-1}$ es triangular inferior, $(L^{-1})^T$ es triangular superior
- Por lo tanto: $A^{-1} = (L^{-1})^T L^{-1}$

**Utilidad en QDA**

Para la forma cuadrática:

En QDA tenemos:

$$(x-\mu_j)^T \Sigma_j^{-1} (x-\mu_j)$$
Si $\Sigma_j = LL^T$, entonces $\Sigma_j^{-1} = (L^{-1})^T L^{-1}$, y podemos escribir:
$$(x-\mu_j)^T (L^{-1})^T L^{-1} (x-\mu_j) = \|L^{-1}(x-\mu_j)\|^2$$
Definiendo $y = L^{-1}(x-\mu_j)$, la forma cuadrática se reduce a:
$$\|y\|^2 = y^Ty = \sum_{i=1}^p y_i^2$$
que es mucho más simple y eficiente de calcular.

Para el determinante:

$$|\Sigma_j^{-1}| = |(L^{-1})^T L^{-1}| = |L^{-1}|^2 = \left(\prod_{i=1}^p (L^{-1})_{ii}\right)^2$$
Por lo tanto:
$$\log|\Sigma_j^{-1}| = 2\log\left(\prod_{i=1}^p (L^{-1}){ii}\right) = 2\sum{i=1}^p \log(L^{-1})_{ii}$$
O equivalentemente, si trabajamos con $L$ en lugar de $L^{-1}$:
$$\log|\Sigma_j^{-1}| = -2\sum_{i=1}^p \log L_{ii}$$

**Ventajas computacionales:**
1. Evita calcular explícitamente $\Sigma_j^{-1}$ (matriz completa $p \times p$)
2. Reduce la forma cuadrática a un producto escalar simple 
3. El determinante se calcula solo con los elementos de la diagonal
4. Aprovecha la estructura triangular de $L$ para operaciones más eficientes

### Enunciado N°9

#### Diferencias entre `QDA` y `QDA_Chol1`

##### Diferencias principales

**`QDA` (implementación directa):**
- Calcula e invierte directamente la matriz de covarianza: `Σ_j^{-1} = LA.inv(np.cov(...))`
- Almacena `inv_covs` (las matrices inversas completas `Σ_j^{-1}`)
- Calcula el determinante de `Σ_j^{-1}` en cada predicción: `LA.det(inv_cov)`
- Forma cuadrática: `unbiased_x.T @ inv_cov @ unbiased_x`

**`QDA_Chol1` (usando descomposición de Cholesky):**
- Factoriza la matriz de covarianza usando Cholesky: `L_j = cholesky(Σ_j, lower=True)` donde `Σ_j = L_j @ L_j^T`
- Invierte la matriz triangular de Cholesky: `L_inv_j = LA.inv(L_j)`
- Almacena `L_invs` (las inversas de las factorizaciones de Cholesky `L_j^{-1}`)
- Calcula el determinante usando la diagonal: `prod(diag(L_inv_j))`
- Forma cuadrática: `||L_inv @ unbiased_x||^2`

##### Paso a paso: Cómo `QDA_Chol1` llega a las predicciones

**Fase 1: Entrenamiento (`_fit_params`)**

Para cada clase `idx = 0, 1, ..., k-1`:

1. **Filtrar datos de la clase**: `X[:,y.flatten()==idx]` (columnas de X correspondientes a clase idx)

2. **Calcular matriz de covarianza**: `Σ_idx = np.cov(X[:,y.flatten()==idx], bias=True)` (matriz p×p)

3. **Descomposición de Cholesky**: `L_idx = cholesky(Σ_idx, lower=True)` donde `Σ_idx = L_idx @ L_idx^T` y `L_idx` es triangular inferior

4. **Invertir la matriz triangular de Cholesky**: `L_inv_idx = LA.inv(L_idx)` (más eficiente que invertir `Σ_idx` directamente)

5. **Calcular media de la clase**: `μ_idx = X[:,y.flatten()==idx].mean(axis=1, keepdims=True)` (vector p×1)

6. **Almacenar en listas**: `self.L_invs = [L_inv_0, L_inv_1, ..., L_inv_{k-1}]` y `self.means = [μ_0, μ_1, ..., μ_{k-1}]`

**Fase 2: Predicción (`_predict_log_conditional`)**

Para una observación `x` y clase candidata `class_idx`:

1. **Recuperar parámetros de la clase**: `L_inv = self.L_invs[class_idx]` (matriz p×p triangular inferior invertida) y `μ = self.means[class_idx]` (vector p×1)

2. **Centrar la observación**: `unbiased_x = x - μ` (vector p×1: `(x - μ_j)`)

3. **Transformar usando `L^{-1}`**: `y = L_inv @ unbiased_x` (vector p×1)

   **Relación matemática clave:** En `QDA` calculábamos la forma cuadrática: `(x-μ_j)^T Σ_j^{-1} (x-μ_j)`.
   
    Como `Σ_j = L_j @ L_j^T`, entonces `Σ_j^{-1} = (L_j^T)^{-1} @ L_j^{-1} = (L_j^{-1})^T @ L_j^{-1}`.
    
    Por lo tanto: `(x-μ_j)^T Σ_j^{-1} (x-μ_j) = (x-μ_j)^T (L_inv^T @ L_inv) (x-μ_j) = (L_inv @ (x-μ_j))^T @ (L_inv @ (x-μ_j)) = ||y||^2`

4. **Calcular log-determinante**: `log_det_term = np.log(L_inv.diagonal().prod())`

   **Justificación:** Para matrices triangulares: `det(L_inv) = prod(diag(L_inv))`. Como `det(Σ_j^{-1}) = det((L_j^{-1})^T @ L_j^{-1}) = det(L_j^{-1})^2`, entonces: `log(|Σ_j^{-1}|^{1/2}) = log(det(L_j^{-1})) = log(prod(diag(L_inv)))`

5. **Calcular distancia de Mahalanobis al cuadrado**: `mahalanobis_sq = (y**2).sum()` que equivale a `y^T @ y = ||y||^2`

6. **Retornar log-densidad condicional**: `return np.log(L_inv.diagonal().prod()) - 0.5 * (y**2).sum()` que corresponde a: `log f_j(x) = (1/2)log|Σ_j^{-1}| - (1/2)(x-μ_j)^T Σ_j^{-1} (x-μ_j) + C`

**Fase 3: Clasificación (heredada de `BaseBayesianClassifier`)**

Para cada observación, el modelo: (1) Calcula `log f_j(x) + log π_j` para todas las clases `j` (en `_predict_one`), (2) Retorna `argmax_j { log f_j(x) + log π_j }`

##### Ventajas de `QDA_Chol1` sobre `QDA`

1. **Estabilidad numérica**: La descomposición de Cholesky es más estable que invertir directamente `Σ_j`
2. **Eficiencia en inversión**: Invertir una matriz triangular (`L_j`) es más rápido que invertir una matriz densa (`Σ_j`)
3. **Determinante eficiente**: El determinante se obtiene como producto de la diagonal (operación O(p) vs O(p³))
4. **Forma cuadrática optimizada**: Calcular `||L_inv @ v||^2` requiere una sola multiplicación matriz-vector seguida de un producto punto, más eficiente que `v^T @ Σ_inv @ v`

### Enunciado N°10

#### Diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`

Las tres implementaciones usan la descomposición de Cholesky pero de formas distintas:

##### `QDA_Chol1` - Invierte L con NumPy

**Entrenamiento:**
```python
L = cholesky(Σ_j)           # Factoriza: Σ = L·L^T
L_inv = LA.inv(L)           # Invierte L usando NumPy
```
**Predicción:**
```python
y = L_inv @ (x - μ_j)       # Multiplica directamente
```
**Ventaja:** simple, usa solo NumPy.

**Desventaja:** LA.inv*() no aprovecha que L es triangular.

##### `QDA_Chol2` - NO invierte, resuelve sistemas

**Entrenamiento:**
```python
L = cholesky(Σ_j)           # Solo factoriza, NO invierte
```
**Predicción:**
```python
y = solve_triangular(L, x - μ_j)  # Resuelve L·y = (x-μ) sin invertir
```
**Ventaja:** Entrenamiento más rápido (no invierte)  

**Desventaja:** Predicción más lenta (resuelve sistema cada vez)

##### `QDA_Chol3` - Invierte L con LAPACK optimizado

**Entrenamiento:**
```python
L = cholesky(Σ_j)           # Factoriza
L_inv = dtrtri(L)           # Invierte usando LAPACK (optimizado para triangulares)\
```
**Predicción:**
```python
y = L_inv @ (x - μ_j)       # Multiplica directamente
```
**Ventaja:** Inversión más eficiente que NumPy, predicción rápida.

**Desventaja:** Entrenamiento más lento que Chol2.

##### En resumen

- **Chol1:** Invierte con NumPy (opción intermedia)
- **Chol2:** No invierte, resuelve cada vez (entrena rápido, predice lento)
- **Chol3:** Invierte con LAPACK (entrena lento, predice rápido)

### Enunciado N°11

In [26]:
#Comparación de las 7 variantes de QDA

# Primero, aseguramos tener el dataset cargado
X_full, y_full = get_wine_dataset()
y_full_encoded = label_encode(y_full)

# Creamos un benchmark con más runs para tener resultados más confiables
b_chol = Benchmark(
    X_full, y_full_encoded,
    n_runs = 500,      # Más runs para mejor precisión
    warmup = 50,
    mem_runs = 50,
    test_sz = 0.3,
    same_splits = True  # Mismo split para comparación justa
)

# Bencheamos las 7 variantes
modelos_a_comparar = [
    QDA,              # 1. Original
    TensorizedQDA,    # 2. Tensorizado sobre clases
    FasterQDA,        # 3. Elimina for en predict
    EfficientQDA,     # 4. Eficiente sin matriz n×n
    QDA_Chol1,        # 5. Cholesky con LA.inv
    QDA_Chol2,        # 6. Cholesky con solve_triangular
    QDA_Chol3         # 7. Cholesky con dtrtri
]

print("Benchmarking 7 variantes de QDA...")
for modelo in modelos_a_comparar:
    b_chol.bench(modelo)

# Generamos el resumen completo
print("\n" + "="*80)
print("RESUMEN COMPLETO DE PERFORMANCE")
print("="*80)
summ_completo = b_chol.summary(baseline='QDA')
print(summ_completo)

# Análisis específico: tiempos de entrenamiento y predicción
print("\n" + "="*80)
print("ANÁLISIS DE TIEMPOS")
print("="*80)
tiempos = summ_completo[[
    'train_median_ms', 'train_std_ms', 
    'test_median_ms', 'test_std_ms',
    'train_speedup', 'test_speedup'
]].round(4)
print(tiempos)

# Análisis específico: uso de memoria
print("\n" + "="*80)
print("ANÁLISIS DE MEMORIA")
print("="*80)
memoria = summ_completo[[
    'train_mem_median_mb', 'train_mem_std_mb',
    'test_mem_median_mb', 'test_mem_std_mb',
    'train_mem_reduction', 'test_mem_reduction'
]].round(4)
print(memoria)

# Análisis específico: accuracy
print("\n" + "="*80)
print("ANÁLISIS DE ACCURACY")
print("="*80)
accuracy = summ_completo[['mean_accuracy']].round(6)
print(accuracy)

# Identificar mejor y peor modelo
print("\n" + "="*80)
print("RANKING DE MODELOS")
print("="*80)

# Mejor en entrenamiento
mejor_train = summ_completo['train_median_ms'].idxmin()
print(f"✓ Más rápido en ENTRENAMIENTO: {mejor_train} ({summ_completo.loc[mejor_train, 'train_median_ms']:.4f} ms)")

# Mejor en predicción
mejor_test = summ_completo['test_median_ms'].idxmin()
print(f"✓ Más rápido en PREDICCIÓN: {mejor_test} ({summ_completo.loc[mejor_test, 'test_median_ms']:.4f} ms)")

# Mejor en memoria de entrenamiento
mejor_mem_train = summ_completo['train_mem_median_mb'].idxmin()
print(f"✓ Menos memoria en ENTRENAMIENTO: {mejor_mem_train} ({summ_completo.loc[mejor_mem_train, 'train_mem_median_mb']:.4f} MB)")

# Mejor en memoria de predicción
mejor_mem_test = summ_completo['test_mem_median_mb'].idxmin()
print(f"✓ Menos memoria en PREDICCIÓN: {mejor_mem_test} ({summ_completo.loc[mejor_mem_test, 'test_mem_median_mb']:.4f} MB)")

# Peor en entrenamiento
peor_train = summ_completo['train_median_ms'].idxmax()
print(f"✗ Más lento en ENTRENAMIENTO: {peor_train} ({summ_completo.loc[peor_train, 'train_median_ms']:.4f} ms)")

# Peor en predicción
peor_test = summ_completo['test_median_ms'].idxmax()
print(f"✗ Más lento en PREDICCIÓN: {peor_test} ({summ_completo.loc[peor_test, 'test_median_ms']:.4f} ms)")

Benching params:
Total runs: 600
Warmup runs: 50
Peak Memory usage runs: 50
Running time runs: 500
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3
Benchmarking 7 variantes de QDA...


QDA_Chol3 (TIME): 100%|██████████| 500/500 [00:00<00:00, 891.83it/s]


RESUMEN COMPLETO DE PERFORMANCE
               train_median_ms  train_std_ms  test_median_ms  test_std_ms  \
model                                                                       
QDA                    0.14270      0.051684         1.21615     0.226336   
TensorizedQDA          0.14480      0.065108         0.56755     0.130810   
FasterQDA              0.13115      0.045885         0.03490     0.029630   
EfficientQDA           0.13255      0.043001         0.03490     0.010151   
QDA_Chol1              0.17240      0.072901         0.72835     0.133840   
QDA_Chol2              0.15330      0.098505         1.80160     0.711904   
QDA_Chol3              0.14175      0.094495         0.73865     0.137910   

               mean_accuracy  train_mem_median_mb  train_mem_std_mb  \
model                                                                 
QDA                 0.984148             0.018160          0.000980   
TensorizedQDA       0.984148             0.018147          0

#### Análisis de Performance de las 7 Variantes de QDA

##### Observaciones Generales

**Accuracy:**
- Todas las variantes mantienen el mismo accuracy (≈98.4%), lo cual es esperado ya que implementan el mismo algoritmo matemático, solo difieren en la implementación computacional.

**Tiempos de Entrenamiento:**
- Las diferencias son pequeñas (~0.04 ms entre la más rápida y más lenta)
- `EfficientQDA` es marginalmente la más rápida (0.1457 ms)
- `QDA_Chol1` es la más lenta (0.1986 ms) porque invierte L con `LA.inv` sin aprovechar que es triangular

**Tiempos de Predicción:**
- Aquí sí hay diferencias **dramáticas** (factor 32x entre extremos)
- Las variantes vectorizadas (`FasterQDA`, `EfficientQDA`) son **~32x más rápidas** que QDA base
- `QDA_Chol2` es la **más lenta** (1.66 ms) porque resuelve un sistema triangular por cada observación

**Uso de Memoria:**
- Entrenamiento: Todas usan memoria similar (~0.018 MB)
- Predicción: Las variantes vectorizadas usan **~10x más memoria** (0.075 MB vs 0.0076 MB) porque procesan todas las observaciones simultáneamente

##### Comparación entre implementaciones de Cholesky

**`QDA_Chol1` (LA.inv sobre L):**
- ✗ Entrenamiento más lento (0.1986 ms)
- ✓ Predicción rápida (0.6934 ms)
- ✗ No aprovecha que L es triangular
- **Conclusión:** No tiene ventajas sobre Chol3

**`QDA_Chol2` (solve_triangular):**
- ✓ Entrenamiento intermedio (0.1690 ms)
- ✗✗ Predicción **MÁS LENTA de todas** (1.6628 ms)
- ✗ Resuelve sistema triangular en cada predicción
- **Conclusión:** Claramente la peor opción

**`QDA_Chol3` (dtrtri):**
- ✓ Entrenamiento rápido (0.1562 ms)
- ✓ Predicción rápida (0.6856 ms)
- ✓ Inversión optimizada para matrices triangulares
- ✓ Pre-calcula la inversa una sola vez
- **Conclusión:** Mejor implementación de Cholesky


### Enunciado N°12

In [27]:
class TensorizedChol(QDA_Chol3):
    def _fit_params(self, X, y):
        # Reutiliza el ajuste de la versión Chol
        super()._fit_params(X, y)

        # Apila en tensores: (K, d, d) y (K, d, 1)
        self.tensor_L_invs  = np.stack(self.L_invs, axis=0)
        self.tensor_means   = np.stack(self.means,  axis=0)

        # Guarda también el logdet-term por clase para no recalcular
        # logdet_term_k = sum(log(diag(L_inv_k)))   # escalar por clase
        self.tensor_logdet_terms = np.sum(
            np.log(np.diagonal(self.tensor_L_invs, axis1=1, axis2=2)),
            axis=1
        )  # shape: (K,)

    def _predict_log_conditionals(self, x):
        """
        Devuelve (K,) con log p(x|k) hasta constante común:
          log p(x|k) = sum(log diag(L_inv_k)) - 0.5 * || L_inv_k (x - mu_k) ||^2
        """
        x = np.asarray(x)
        if x.ndim == 1:
            x = x[:, None]  # (d,1)

        # (K, d, 1): resta por clase
        unbiased = x[None, :, :] - self.tensor_means

        # y_k = L_inv_k @ unbiased_k  -> (K, d, 1)
        y = np.einsum('kij,kjn->kin', self.tensor_L_invs, unbiased)

        quad = np.sum(y**2, axis=(1, 2))  # (K,)

        return self.tensor_logdet_terms - 0.5 * quad  # (K,)

    def _predict_one(self, x):
        return int(np.argmax(self.log_a_priori + self._predict_log_conditionals(x)))

### Enunciado N°13

In [28]:
class EfficientChol(TensorizedChol):
    def predict(self, X):
        """
        X: (d, n)  -> devuelve (n,) clases predichas
        log p(x_j|k) = sum(log diag(L_inv_k)) - 0.5 * || L_inv_k (x_j - mu_k) ||^2
        """
        X = np.asarray(X)
        if X.ndim == 1:
            X = X[:, None]  # (d,1)

        # (K, d, n): resta por clase (evita formar n×n)
        unbiased = X[None, :, :] - self.tensor_means

        # y = L_inv_k @ (X - mu_k)  -> (K, d, n)
        y = np.einsum('kij,kjn->kin', self.tensor_L_invs, unbiased)

        # ||y||^2 por clase y muestra usando suma de Hadamard por columnas (punto 5)
        quad = np.sum(y * y, axis=1)                 # (K, n)

        # log-condicionales y posterior
        log_conditionals = self.tensor_logdet_terms[:, None] - 0.5 * quad   # (K, n)
        log_post = self.log_a_priori[:, None] + log_conditionals            # (K, n)

        return np.argmax(log_post, axis=0)  # (n,)

### Enunciado N°14

In [29]:
#Comparación de las 9 variantes de QDA

# Primero, aseguramos tener el dataset cargado
X_full, y_full = get_wine_dataset()
y_full_encoded = label_encode(y_full)

# Creamos un benchmark con más runs para tener resultados más confiables
b_chol = Benchmark(
    X_full, y_full_encoded,
    n_runs = 500,      # Más runs para mejor precisión
    warmup = 50,
    mem_runs = 50,
    test_sz = 0.3,
    same_splits = True  # Mismo split para comparación justa
)

# Bencheamos las 9 variantes
modelos_a_comparar = [
    QDA,              # 1. Original
    TensorizedQDA,    # 2. Tensorizado sobre clases
    FasterQDA,        # 3. Elimina for en predict
    EfficientQDA,     # 4. Eficiente sin matriz n×n
    QDA_Chol1,        # 5. Cholesky con LA.inv
    QDA_Chol2,        # 6. Cholesky con solve_triangular
    QDA_Chol3,         # 7. Cholesky con dtrtri
    TensorizedChol,  # 8. Tensorizado con Cholesky
    EfficientChol    # 9. Eficiente con Cholesky
]

print("Benchmarking 9 variantes de QDA...")
for modelo in modelos_a_comparar:
    b_chol.bench(modelo)

# Generamos el resumen completo
print("\n" + "="*80)
print("RESUMEN COMPLETO DE PERFORMANCE")
print("="*80)
summ_completo = b_chol.summary(baseline='QDA')
print(summ_completo)

# Análisis específico: tiempos de entrenamiento y predicción
print("\n" + "="*80)
print("ANÁLISIS DE TIEMPOS")
print("="*80)
tiempos = summ_completo[[
    'train_median_ms', 'train_std_ms', 
    'test_median_ms', 'test_std_ms',
    'train_speedup', 'test_speedup'
]].round(4)
print(tiempos)

# Análisis específico: uso de memoria
print("\n" + "="*80)
print("ANÁLISIS DE MEMORIA")
print("="*80)
memoria = summ_completo[[
    'train_mem_median_mb', 'train_mem_std_mb',
    'test_mem_median_mb', 'test_mem_std_mb',
    'train_mem_reduction', 'test_mem_reduction'
]].round(4)
print(memoria)

# Análisis específico: accuracy
print("\n" + "="*80)
print("ANÁLISIS DE ACCURACY")
print("="*80)
accuracy = summ_completo[['mean_accuracy']].round(6)
print(accuracy)

# Identificar mejor y peor modelo
print("\n" + "="*80)
print("RANKING DE MODELOS")
print("="*80)

# Mejor en entrenamiento
mejor_train = summ_completo['train_median_ms'].idxmin()
print(f"✓ Más rápido en ENTRENAMIENTO: {mejor_train} ({summ_completo.loc[mejor_train, 'train_median_ms']:.4f} ms)")

# Mejor en predicción
mejor_test = summ_completo['test_median_ms'].idxmin()
print(f"✓ Más rápido en PREDICCIÓN: {mejor_test} ({summ_completo.loc[mejor_test, 'test_median_ms']:.4f} ms)")

# Mejor en memoria de entrenamiento
mejor_mem_train = summ_completo['train_mem_median_mb'].idxmin()
print(f"✓ Menos memoria en ENTRENAMIENTO: {mejor_mem_train} ({summ_completo.loc[mejor_mem_train, 'train_mem_median_mb']:.4f} MB)")

# Mejor en memoria de predicción
mejor_mem_test = summ_completo['test_mem_median_mb'].idxmin()
print(f"✓ Menos memoria en PREDICCIÓN: {mejor_mem_test} ({summ_completo.loc[mejor_mem_test, 'test_mem_median_mb']:.4f} MB)")

# Peor en entrenamiento
peor_train = summ_completo['train_median_ms'].idxmax()
print(f"✗ Más lento en ENTRENAMIENTO: {peor_train} ({summ_completo.loc[peor_train, 'train_median_ms']:.4f} ms)")

# Peor en predicción
peor_test = summ_completo['test_median_ms'].idxmax()
print(f"✗ Más lento en PREDICCIÓN: {peor_test} ({summ_completo.loc[peor_test, 'test_median_ms']:.4f} ms)")

Benching params:
Total runs: 600
Warmup runs: 50
Peak Memory usage runs: 50
Running time runs: 500
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3
Benchmarking 9 variantes de QDA...


EfficientChol (TIME): 100%|██████████| 500/500 [00:00<00:00, 2325.49it/s]


RESUMEN COMPLETO DE PERFORMANCE
                train_median_ms  train_std_ms  test_median_ms  test_std_ms  \
model                                                                        
QDA                     0.15230      0.072848         1.23525     0.193241   
TensorizedQDA           0.15135      0.085244         0.56330     0.176048   
FasterQDA               0.13470      0.042582         0.03580     0.012949   
EfficientQDA            0.13340      0.051266         0.03550     0.016478   
QDA_Chol1               0.17820      0.068673         0.74985     0.164019   
QDA_Chol2               0.16635      0.085405         1.87150     0.420975   
QDA_Chol3               0.15280      0.103205         0.75940     0.271202   
TensorizedChol          0.16250      0.115744         0.40485     0.238259   
EfficientChol           0.15645      0.141970         0.04355     0.036973   

                mean_accuracy  train_mem_median_mb  train_mem_std_mb  \
model                               

#### Análisis de Performance de las 9 Variantes de QDA

##### Observaciones Generales

**Accuracy:**
- Todas las variantes mantienen el mismo accuracy (≈98.4%), lo cual es esperado ya que implementan el mismo algoritmo matemático, solo difieren en la implementación computacional.

**Tiempos de Entrenamiento:**
- Las diferencias son pequeñas (~0.05 ms entre la más rápida y más lenta)
- `QDA_Chol3` y `EfficientChol` son marginalmente las más rápida (0.15530 y 0.1605 ms, respectivamente)
- `QDA_Chol1` es la más lenta (0.217 ms)

**Tiempos de Predicción:**
- Aquí sí hay diferencias **significativas**
- Las variantes vectorizadas (`FasterQDA`, `EfficientQDA`, `EfficientChol`) son **+ 30x más rápidas** que QDA_Chol2
- `QDA_Chol2` es la **más lenta** (1.9 ms) porque resuelve un sistema triangular por cada observación

**Uso de Memoria:**
- Entrenamiento: Todas usan memoria similar (~0.018 MB)
- Predicción: EfficientChol usa **~3.5x más memoria** (0.2307 MB) que las variantes vectorizadas (`FasterQDA`, `EfficientQDA`)


**¿Se condice con lo esperado?**

En general, sí:

* Las versiones “Efficient” evitan construir matrices 𝑛 × 𝑛 y se benefician de GEMM/einsum → por eso EfficientQDA y EfficientChol vuelan en predicción.

* Si comparamos las variantes QDA_Cholesky con las mismas combinadas con Tensorized y la version "Efficient", la reducción en el tiempo de predicción en ampliamente notoria (EfficientChol: 0.055 ms vs QDA_Chol1: 0.9 ms, QDA_Chol2: 1.9 ms, QDA_Chol3:0.84 ms). No obstante, consume más memoria.

* Con Cholesky, si no se aprovecha la triangularidad con solve_triangular batched (o equivalente), se termina multiplicando con L^(-1) como si fuera densa. Eso explica por qué EfficientChol mejora bastante, pero no supera a EfficientQDA (NumPy/BLAS tiene GEMM ultra optimizado; la ruta triangular no siempre gana si no hay TRSM batched).


### Respuestas a las preguntas del Codigo

* para que sirve bincount?

Cuenta cuántas veces aparece cada valor en Y, retorna un array y despues al dividir por y.size se convierte en probabilidades a priori

* por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?

El metodo fit necesita que las clases esten encodeadas y al utilizar el atributo self.log_a_priori si no se tiene el estimate_a_priori va a romper y para obtener esas probabilidades a priori deben estar encodeadas las clases

* por que hace falta el flatten y no se puede directamente X[:,y==idx]?

Sin flatten NumPy intenta indexar con una matriz 2D en lugar de un vector 1D por lo cual arroja error

* por que se usa bias=True en vez del default bias=False?

El bias= True es el estimador de máxima verosimilitud divide n en lugar de n-1 (que es el valor por default)

* que hace axis=1? por que no axis=0?

En QDA se necesita un vector de medias por feature, es decir una media para cada dimensión axis = 1 tiene forma (n_features, 1) donde una matriz X de dimensiones (n, m) indica que tiene n features y hay m observaciones